# 外交青書2025 テキストマイニング分析（R版）

## 概要

このノートブックでは、外務省が発行する「外交青書2025」第1章「国際情勢認識と日本外交の展望」のPDFファイルに対して、Rを使用したテキストマイニング分析を行います。

### 分析内容

1. **PDFからのテキスト抽出** - pdftoolsを使用してPDFからテキストデータを抽出
2. **形態素解析** - RMeCabを使用して日本語テキストを単語に分解
3. **単語頻度分析** - 出現頻度の高い単語を特定
4. **共起分析** - 同時に出現する単語ペアを分析
5. **ワードクラウド生成** - 分析結果を視覚化

### 使用パッケージ

| パッケージ | 用途 |
|-----------|------|
| pdftools | PDFからテキスト抽出 |
| RMeCab | 日本語形態素解析 |
| dplyr | データ操作 |
| ggplot2 | グラフ描画 |
| wordcloud2 | ワードクラウド生成 |
| igraph | ネットワーク可視化 |

---
## 1. 環境設定

必要なパッケージをインストール・読み込みます。

### 前提条件
- MeCabがシステムにインストールされていること
- `sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8`

In [ ]:
# パッケージのインストール（未インストールの場合）
packages <- c("pdftools", "dplyr", "tidyr", "ggplot2", "stringr", 
              "wordcloud2", "igraph", "widyr", "tidytext")

for (pkg in packages) {
  if (!require(pkg, character.only = TRUE, quietly = TRUE)) {
    install.packages(pkg, repos = "https://cran.r-project.org")
  }
}

# RMeCabのインストール（特殊な処理が必要）
if (!require("RMeCab", quietly = TRUE)) {
  install.packages("RMeCab", repos = "https://rmecab.jp/R", type = "source")
}

In [ ]:
# パッケージの読み込み
library(pdftools)
library(dplyr)
library(tidyr)
library(ggplot2)
library(stringr)
library(RMeCab)

# 日本語フォント設定
theme_set(theme_gray(base_family = "Noto Sans CJK JP"))

cat("パッケージの読み込み完了\n")

---
## 2. 設定とパラメータ

分析に使用する各種パラメータを定義します。

In [ ]:
# ===== ファイル設定 =====
PDF_PATH <- "1_2_1.pdf"
OUTPUT_CSV <- "word_frequency_results_r.csv"
COOCCURRENCE_CSV <- "cooccurrence_results_r.csv"

# ===== 形態素解析の設定 =====
# 抽出対象の品詞
TARGET_POS <- c("名詞", "動詞", "形容詞")

# ストップワード（除外する一般的な単語）
STOPWORDS <- c(
  # 形式名詞
  "こと", "もの", "ため", "よう",
  # 代名詞
  "これ", "それ", "あれ", "ここ", "そこ", "あそこ",
  "どこ", "どれ", "なに", "何",
  # 補助動詞・基本動詞
  "する", "いる", "ある", "なる", "れる", "られる", "せる",
  "できる", "おる", "くる", "来る", "行く", "いく",
  # 指示詞
  "この", "その", "あの", "どの",
  # 否定
  "ない", "なく",
  # 接尾辞
  "等", "的", "化", "性", "上", "中", "下", "内", "外",
  # 時間・番号
  "年", "月", "日", "号", "第", "章",
  # 接続詞等
  "ほか", "また", "および", "かつ", "ただし", "なお", "または"
)

cat("対象PDF:", PDF_PATH, "\n")
cat("抽出品詞:", paste(TARGET_POS, collapse = ", "), "\n")
cat("ストップワード数:", length(STOPWORDS), "語\n")

---
## 3. PDFからテキスト抽出

pdftoolsを使用してPDFファイルからテキストを抽出します。

In [ ]:
# PDFからテキストを抽出
cat("PDFからテキストを抽出中...\n\n")

pdf_text <- pdf_text(PDF_PATH)
cat("総ページ数:", length(pdf_text), "\n")

for (i in seq_along(pdf_text)) {
  cat(sprintf("  ページ %d: %d 文字抽出\n", i, nchar(pdf_text[i])))
}

# 全ページを結合
raw_text <- paste(pdf_text, collapse = "\n")
cat(sprintf("\n合計抽出文字数: %s 文字\n", format(nchar(raw_text), big.mark = ",")))

In [ ]:
# 抽出したテキストの先頭部分を確認
cat(strrep("=", 60), "\n")
cat("抽出テキスト（先頭500文字）\n")
cat(strrep("=", 60), "\n")
cat(substr(raw_text, 1, 500), "\n")

---
## 4. テキストの前処理（クリーニング）

形態素解析の精度を高めるため、テキストから不要な文字を除去します。

In [ ]:
clean_text <- function(text) {
  # 数字を除去（半角・全角）
  text <- str_replace_all(text, "[0-9０-９]+", "")
  
  # 英字を除去（半角・全角）
  text <- str_replace_all(text, "[a-zA-Zａ-ｚＡ-Ｚ]+", "")
  
  # 記号・空白を除去
  text <- str_replace_all(text, "[（）()【】「」『』・、。：；！？]+", " ")
  
  # 連続する空白を1つに
  text <- str_replace_all(text, "\\s+", " ")
  
  return(str_trim(text))
}

# クリーニングの実行
cleaned_text <- clean_text(raw_text)

cat(sprintf("クリーニング前: %s 文字\n", format(nchar(raw_text), big.mark = ",")))
cat(sprintf("クリーニング後: %s 文字\n", format(nchar(cleaned_text), big.mark = ",")))
cat(sprintf("削減率: %.1f%%\n", (1 - nchar(cleaned_text) / nchar(raw_text)) * 100))

---
## 5. 形態素解析

RMeCabを使用して日本語テキストを形態素（単語）に分解します。

In [ ]:
# 形態素解析を実行
cat("形態素解析を実行中...\n")

# テキストを一時ファイルに保存してRMeCabFreqで解析
temp_file <- tempfile(fileext = ".txt")
writeLines(cleaned_text, temp_file, useBytes = TRUE)

# 形態素解析（品詞情報付き）
mecab_result <- RMeCabFreq(temp_file)

# 結果をデータフレームに変換
words_df <- as.data.frame(mecab_result) %>%
  filter(Info1 %in% TARGET_POS) %>%           # 指定品詞のみ
  filter(!(Info2 %in% c("非自立", "代名詞", "数"))) %>%  # サブカテゴリ除外
  filter(nchar(Term) > 1) %>%                  # 1文字の単語を除外
  filter(!(Term %in% STOPWORDS)) %>%          # ストップワードを除外
  arrange(desc(Freq))

# 一時ファイルを削除
unlink(temp_file)

cat(sprintf("\n抽出単語数: %s 語\n", format(sum(words_df$Freq), big.mark = ",")))
cat(sprintf("ユニーク単語数: %s 語\n", format(nrow(words_df), big.mark = ",")))

In [ ]:
# 形態素解析の結果サンプルを表示
cat("抽出された単語（上位20語）:\n")
head(words_df, 20)

---
## 6. 単語頻度分析

抽出した単語の出現回数を分析し、可視化します。

In [ ]:
# 上位30語を表示
TOP_N <- 30

cat(strrep("=", 60), "\n")
cat(sprintf("単語頻度ランキング（上位%d語）\n", TOP_N))
cat(strrep("=", 60), "\n")

top_words <- head(words_df, TOP_N)
max_freq <- max(top_words$Freq)

for (i in 1:nrow(top_words)) {
  word <- top_words$Term[i]
  freq <- top_words$Freq[i]
  bar_len <- round(freq / max_freq * 30)
  bar <- paste(rep("█", bar_len), collapse = "")
  cat(sprintf("%2d. %-12s %3d %s\n", i, word, freq, bar))
}

In [ ]:
# 棒グラフで可視化
top20 <- head(words_df, 20)

ggplot(top20, aes(x = reorder(Term, Freq), y = Freq)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  coord_flip() +
  labs(title = "単語頻度ランキング（上位20語）",
       x = "単語",
       y = "出現回数") +
  theme_minimal(base_family = "Noto Sans CJK JP") +
  theme(axis.text.y = element_text(size = 12))

---
## 7. 共起分析

同時に出現する単語ペアを分析します。

In [ ]:
# 文単位で共起分析
cat("共起分析を実行中...\n")

# テキストを文に分割
sentences <- unlist(str_split(raw_text, "[。！？\n]+"))
sentences <- sentences[nchar(sentences) > 10]  # 短すぎる文を除外

cat(sprintf("文の数: %d\n", length(sentences)))

# 上位50語のみを共起分析対象とする
top50_words <- head(words_df$Term, 50)

# 各文から単語を抽出して共起をカウント
cooccurrence_list <- list()

for (sent in sentences) {
  # 文を形態素解析
  temp_sent <- tempfile(fileext = ".txt")
  writeLines(sent, temp_sent, useBytes = TRUE)
  
  tryCatch({
    sent_result <- RMeCabFreq(temp_sent)
    sent_df <- as.data.frame(sent_result)
    
    # 上位50語に含まれる単語のみ抽出
    sent_words <- unique(sent_df$Term[sent_df$Term %in% top50_words])
    
    # 単語ペアを作成
    if (length(sent_words) >= 2) {
      pairs <- combn(sort(sent_words), 2, simplify = FALSE)
      for (pair in pairs) {
        key <- paste(pair, collapse = "-")
        if (is.null(cooccurrence_list[[key]])) {
          cooccurrence_list[[key]] <- 0
        }
        cooccurrence_list[[key]] <- cooccurrence_list[[key]] + 1
      }
    }
  }, error = function(e) {})
  
  unlink(temp_sent)
}

# 共起データフレームを作成
cooccurrence_df <- data.frame(
  pair = names(cooccurrence_list),
  count = unlist(cooccurrence_list),
  stringsAsFactors = FALSE
) %>%
  arrange(desc(count)) %>%
  separate(pair, into = c("word1", "word2"), sep = "-")

cat(sprintf("\n共起ペア数: %s ペア\n", format(nrow(cooccurrence_df), big.mark = ",")))

In [ ]:
# 共起頻度上位20ペアを表示
cat(strrep("=", 50), "\n")
cat("共起頻度ランキング（上位20ペア）\n")
cat(strrep("=", 50), "\n")

top20_pairs <- head(cooccurrence_df, 20)
for (i in 1:nrow(top20_pairs)) {
  cat(sprintf("%2d. %s - %s: %d回\n", 
              i, 
              top20_pairs$word1[i], 
              top20_pairs$word2[i], 
              top20_pairs$count[i]))
}

In [ ]:
# 共起ネットワークの可視化
library(igraph)

# 上位40ペアでネットワーク作成
top_pairs <- head(cooccurrence_df, 40) %>%
  filter(count >= 2)

if (nrow(top_pairs) > 0) {
  # グラフを作成
  g <- graph_from_data_frame(top_pairs[, c("word1", "word2", "count")], directed = FALSE)
  E(g)$weight <- top_pairs$count
  
  # ノードサイズを次数に基づいて設定
  V(g)$size <- degree(g) * 3 + 10
  
  # エッジの太さを重みに基づいて設定
  E(g)$width <- E(g)$weight * 0.5
  
  # プロット
  par(mar = c(0, 0, 2, 0))
  plot(g,
       layout = layout_with_fr(g),
       vertex.color = "lightblue",
       vertex.frame.color = "darkblue",
       vertex.label.family = "Noto Sans CJK JP",
       vertex.label.cex = 0.9,
       vertex.label.color = "black",
       edge.color = "gray70",
       main = "共起ネットワーク")
  
  cat(sprintf("\nノード数: %d\n", vcount(g)))
  cat(sprintf("エッジ数: %d\n", ecount(g)))
}

---
## 8. 品詞別分析

テキスト全体の品詞構成を分析します。

In [ ]:
# 品詞別の出現回数
temp_file <- tempfile(fileext = ".txt")
writeLines(cleaned_text, temp_file, useBytes = TRUE)
all_mecab <- RMeCabFreq(temp_file)
unlink(temp_file)

pos_df <- as.data.frame(all_mecab) %>%
  group_by(Info1) %>%
  summarise(count = sum(Freq), .groups = "drop") %>%
  arrange(desc(count)) %>%
  mutate(percentage = count / sum(count) * 100)

cat(strrep("=", 40), "\n")
cat("品詞別出現回数\n")
cat(strrep("=", 40), "\n")

head(pos_df, 10)

In [ ]:
# 円グラフで可視化
top_pos <- head(pos_df, 6)

ggplot(top_pos, aes(x = "", y = count, fill = Info1)) +
  geom_bar(stat = "identity", width = 1) +
  coord_polar("y", start = 0) +
  labs(title = "品詞構成", fill = "品詞") +
  theme_minimal(base_family = "Noto Sans CJK JP") +
  theme(axis.text = element_blank(),
        axis.title = element_blank(),
        panel.grid = element_blank()) +
  geom_text(aes(label = sprintf("%.1f%%", percentage)),
            position = position_stack(vjust = 0.5))

---
## 9. ワードクラウド生成

単語の出現頻度を視覚的に表現するワードクラウドを生成します。

In [ ]:
# wordcloud2を使用したワードクラウド
library(wordcloud2)

# データを準備
wc_data <- words_df %>%
  select(word = Term, freq = Freq) %>%
  head(100)

# ワードクラウドを生成
wordcloud2(wc_data, 
           size = 0.8,
           color = "random-dark",
           backgroundColor = "white",
           fontFamily = "Noto Sans CJK JP")

---
## 10. 結果の保存

分析結果をCSVファイルに保存します。

In [ ]:
# 単語頻度結果を保存
output_words <- words_df %>%
  mutate(rank = row_number()) %>%
  select(順位 = rank, 単語 = Term, 品詞 = Info1, 出現回数 = Freq)

write.csv(output_words, OUTPUT_CSV, row.names = FALSE, fileEncoding = "UTF-8")
cat(sprintf("単語頻度結果を保存しました: %s\n", OUTPUT_CSV))
cat(sprintf("保存単語数: %d 語\n", nrow(output_words)))

In [ ]:
# 共起分析結果を保存
output_cooc <- cooccurrence_df %>%
  mutate(rank = row_number()) %>%
  select(順位 = rank, 単語1 = word1, 単語2 = word2, 共起回数 = count)

write.csv(output_cooc, COOCCURRENCE_CSV, row.names = FALSE, fileEncoding = "UTF-8")
cat(sprintf("共起分析結果を保存しました: %s\n", COOCCURRENCE_CSV))
cat(sprintf("保存ペア数: %d ペア\n", nrow(output_cooc)))

---
## 11. 分析結果のまとめ

外交青書2025 第1章の分析結果をまとめます。

In [ ]:
# 分析結果のサマリー
cat(strrep("=", 60), "\n")
cat("分析結果サマリー（R版）\n")
cat(strrep("=", 60), "\n")

cat("\n【基本統計】\n")
cat(sprintf("  抽出文字数: %s 文字\n", format(nchar(raw_text), big.mark = ",")))
cat(sprintf("  総単語数: %s 語\n", format(sum(words_df$Freq), big.mark = ",")))
cat(sprintf("  ユニーク単語数: %s 語\n", format(nrow(words_df), big.mark = ",")))
cat(sprintf("  共起ペア数: %s ペア\n", format(nrow(cooccurrence_df), big.mark = ",")))

cat("\n【上位10語】\n")
for (i in 1:min(10, nrow(words_df))) {
  cat(sprintf("  %2d. %s: %d回\n", i, words_df$Term[i], words_df$Freq[i]))
}

cat("\n【共起頻度 上位5ペア】\n")
for (i in 1:min(5, nrow(cooccurrence_df))) {
  cat(sprintf("  %d. %s - %s: %d回\n", 
              i, 
              cooccurrence_df$word1[i], 
              cooccurrence_df$word2[i], 
              cooccurrence_df$count[i]))
}

cat("\n【出力ファイル】\n")
cat(sprintf("  - %s: 単語頻度データ\n", OUTPUT_CSV))
cat(sprintf("  - %s: 共起分析データ\n", COOCCURRENCE_CSV))

cat("\n", strrep("=", 60), "\n")
cat("分析完了\n")
cat(strrep("=", 60), "\n")

---
## 付録: セッション情報

In [ ]:
sessionInfo()